Rede neural CNN treinado sob dados brutos de forma binária

Importar as bibliotecas e os dados do conjunto MNIST

In [1]:
import json
import numpy as np
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist
from tensorflow.keras.callbacks import EarlyStopping

# 1. Carregar o conjunto de dados MNIST
(x_train, y_train), (x_test, y_test) = mnist.load_data()
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
#print (x_train[0][1]) # [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
#print(x_test)

(60000, 28, 28)
(10000, 28, 28)
(60000,)


Normalizar e redimensionar os dados

In [2]:
x_train = x_train.reshape((60000, 28, 28, 1)).astype('float32') / 255
x_test = x_test.reshape((10000, 28, 28, 1)).astype('float32') / 255
print(x_train.shape)

(60000, 28, 28, 1)


Criar os filtros de classes

In [3]:
# Filtrar apenas duas classes (por exemplo, 0 e 1)
binary_classes = [0, 1]
train_filter = np.where((y_train == binary_classes[0]) | (y_train == binary_classes[1]))
test_filter = np.where((y_test == binary_classes[0]) | (y_test == binary_classes[1]))

Filtrar as classes

In [4]:
x_train, y_train = x_train[train_filter], y_train[train_filter]
x_test, y_test = x_test[test_filter], y_test[test_filter]

**Construir o modelo da CNN**

In [5]:
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

c:\Users\mtoma\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Compilar o modelo - https://www.tensorflow.org/api_docs/python/tf/keras/Model#compile

Utiliza o otimizador "adam" - https://www.tensorflow.org/api_docs/python/tf/keras/optimizers

Este código configura o modelo para o treinamento, especificando o otimizador, a função de perda e as métricas a serem avaliadas durante o treinamento e a validação:

optimizer='adam': Define o otimizador Adam para ajustar os pesos da rede. Adam é um algoritmo de otimização baseado em gradiente que é eficiente em termos de computação e tem um requisito de memória relativamente baixo. É amplamente usado por ser eficaz em uma ampla gama de problemas de aprendizado de máquina.

loss='binary_crossentropy': Especifica a função de perda como entropia cruzada binária. Esta é uma escolha comum para problemas de classificação binária. A entropia cruzada binária mede o desempenho do modelo cuja saída é um valor de probabilidade entre 0 e 1. Ela compara a distribuição de probabilidade prevista pela rede com a distribuição real (os rótulos verdadeiros).

metrics=['accuracy']: Define a métrica de avaliação do modelo como precisão (accuracy). A precisão é a fração de previsões corretas entre o total de previsões feitas. É uma métrica comum para avaliar o desempenho de modelos em tarefas de classificação.

Ao chamar model.compile(), você está preparando o modelo para o treinamento, compilando-o com as configurações especificadas. Isso inclui preparar as estruturas de dados internas do modelo e se preparar para a otimização (ajuste dos pesos da rede durante o treinamento).

In [6]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

- Criando e salvando os hiperparâmetros da arquitetura e da inicialização em formato json em arquivos-brutos/hiperparametros.json

- Salvando os pesos iniciais

In [7]:
# Definir os hiperparâmetros da arquitetura da rede neural e os hiperparâmetros de inicialização
hiperparametros = {
    'arquitetura': {
        'camadas': [
            {'tipo': 'Conv2D', 'filtros': 32, 'kernel_size': (3, 3), 'activation': 'relu', 'input_shape': (28, 28, 1)},
            {'tipo': 'MaxPooling2D', 'pool_size': (2, 2)},
            {'tipo': 'Conv2D', 'filtros': 64, 'kernel_size': (3, 3), 'activation': 'relu'},
            {'tipo': 'MaxPooling2D', 'pool_size': (2, 2)},
            {'tipo': 'Conv2D', 'filtros': 64, 'kernel_size': (3, 3), 'activation': 'relu'},
            {'tipo': 'Flatten'},
            {'tipo': 'Dense', 'unidades': 64, 'activation': 'relu'},
            {'tipo': 'Dense', 'unidades': 1, 'activation': 'sigmoid'}
        ]
    },
    'inicializacao': {
        'optimizer': 'adam',
        'loss': 'binary_crossentropy',
        'metrics': ['accuracy']
    }
}

# Serializando os hiperparâmetros em uma string JSON
hiperparametros_json = json.dumps(hiperparametros, indent=4)

# Escrevendo a string JSON em um arquivo
with open("arquivos-brutos/hiperparametros.json", "w") as arquivo:
    arquivo.write(hiperparametros_json)


# PESOS INICIAIS
model.save_weights('arquivos-brutos/pesos_iniciais.weights.h5')

Configurando o Early Stopping

Early Stopping: Monitore o desempenho da rede no conjunto de validação após cada época. Se o desempenho não melhorar após um determinado número de épocas (por exemplo, 10), interrompa o treinamento. Isso ajuda a prevenir overfitting.

In [8]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5)

**Treinar o modelo**

Este trecho de código inicia o treinamento do modelo com os dados fornecidos, especificando também como o treinamento deve ser realizado. Aqui está o que cada argumento significa:

x_train: Os dados de entrada para treinar o modelo. Estes são os exemplos que o modelo usará para aprender.

y_train: As etiquetas (rótulos) correspondentes aos dados de entrada x_train. Estas são as respostas corretas que o modelo tentará prever.

epochs=15: Define o número de épocas de treinamento. Uma época significa uma iteração sobre todos os dados de entrada. Portanto, o modelo passará pelos dados de treinamento 15 vezes.

batch_size=64: Especifica o tamanho do lote (batch size). Isso significa que 64 exemplos de x_train e y_train serão usados para atualizar os pesos do modelo de uma vez. O uso de lotes ajuda a acelerar o treinamento e pode contribuir para uma melhor generalização do modelo.

validation_split=0.2: Este argumento reserva 20% dos dados de treinamento para validação. O modelo não treinará nesses dados. Em vez disso, após cada época, ele avaliará seu desempenho nesse subconjunto de validação. Isso é útil para monitorar o overfitting (quando o modelo aprende padrões específicos dos dados de treinamento, mas falha em generalizar para novos dados).

Portanto, este código treina o modelo nos dados fornecidos (x_train e y_train), por 15 épocas, usando lotes de 64 exemplos, e avalia o desempenho do modelo em 20% dos dados de treinamento reservados para validação após cada época.

In [9]:
historico = model.fit(x_train, y_train, epochs=10, validation_split=0.2, callbacks=[early_stopping])

Epoch 1/10
317/317 ━━━━━━━━━━━━━━━━━━━━ 7s 14ms/step - accuracy: 0.9863 - loss: 0.0763 - val_accuracy: 0.9992 - val_loss: 0.0022
Epoch 2/10
317/317 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - accuracy: 0.9985 - loss: 0.0038 - val_accuracy: 0.9996 - val_loss: 0.0016
Epoch 3/10
317/317 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - accuracy: 0.9991 - loss: 0.0026 - val_accuracy: 1.0000 - val_loss: 2.1151e-04
Epoch 4/10
317/317 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - accuracy: 0.9988 - loss: 0.0023 - val_accuracy: 1.0000 - val_loss: 2.6529e-04
Epoch 5/10
317/317 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 0.9999 - loss: 3.4960e-04 - val_accuracy: 0.9996 - val_loss: 4.1786e-04
Epoch 6/10
317/317 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 1.0000 - loss: 3.0938e-05 - val_accuracy: 1.0000 - val_loss: 1.8687e-04
Epoch 7/10
317/317 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - accuracy: 1.0000 - loss: 4.7634e-06 - val_accuracy: 1.0000 - val_loss: 2.2688e-04
Epoch 8/10
317/317 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 

- Salvando os pesos finais da rede 
- Salvando o histórico de perda para cada iteração
- Salvando as saídas produzidas pela rede para cada um dos dados de teste

In [10]:
# PESOS FINAIS
model.save_weights('arquivos-brutos/pesos_finais.weights.h5')

# ERRO DE CADA ITERAÇÃO
perdas = historico.history['loss']

# Salvando o histórico de perda em um arquivo JSON
with open('arquivos-brutos/historico_perda.json', 'w') as f:
    json.dump(perdas, f)


# SAÍDAS PRODUZIDAS
# Fazendo inferência com o modelo treinado para obter as saídas
saidas = model.predict(x_train)

# Convertendo as saídas para uma lista para serialização
saidas_lista = saidas.tolist()

# Salvando as saídas em um arquivo JSON
with open('arquivos-brutos/saidas_teste.json', 'w') as f:
    json.dump(saidas_lista, f)

396/396 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step


Testar o modelo

In [11]:
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f'Test accuracy: {test_acc}, Test loss: {test_loss}')

67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 1.0000 - loss: 1.6968e-04
Test accuracy: 0.9995272159576416, Test loss: 0.0019101708894595504
